In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

RAW_DIR = PROJECT_ROOT/"data"/"raw"
PROCESSED_DIR = PROJECT_ROOT/"data"/"processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
spot = pd.read_csv(RAW_DIR / "nifty_spot_raw.csv", parse_dates=["timestamp"])
futures = pd.read_csv(RAW_DIR / "nifty_futures.csv", parse_dates=["timestamp"])
options = pd.read_csv(RAW_DIR / "nifty_options_raw.csv", parse_dates=["timestamp"])

print("RAW SHAPES")
print("Spot:", spot.shape)
print("Futures:", futures.shape)
print("Options:", options.shape)

RAW SHAPES
Spot: (19912, 6)
Futures: (19912, 8)
Options: (198130, 8)


In [4]:
full_index = pd.date_range(
    start=spot["timestamp"].min(),
    end=spot["timestamp"].max(),
    freq="5min"
)

In [5]:
spot = (
    spot
    .set_index("timestamp")
    .reindex(full_index)
    .sort_index()
)

# Forward-fill OHLC
ohlc_cols = ["spot_open", "spot_high", "spot_low", "spot_close"]
spot[ohlc_cols] = spot[ohlc_cols].ffill()

# Volume: missing → 0 (no trades)
spot["spot_volume"] = spot["spot_volume"].fillna(0)

# Outlier removal using returns
returns = spot["spot_close"].pct_change()
z = (returns - returns.mean()) / returns.std()

spot = spot[(z.abs() < 5) | (z.isna())]

spot = spot.reset_index().rename(columns={"index": "timestamp"})


In [6]:
futures = (
    futures
    .set_index("timestamp")
    .reindex(full_index)
    .sort_index()
)
 

futures["fut_open_interest"] = futures["fut_open_interest"].ffill()

spot_close_map = (
    spot.set_index("timestamp")["spot_close"]
)

for col in ["fut_open", "fut_high", "fut_low", "fut_close"]:
    futures[col] = futures[col].ffill()

futures = futures.reset_index().rename(columns={"index": "timestamp"})


In [7]:
options = options[options["opt_iv"].notna()]
spot_close_map = (
    spot.set_index("timestamp")["spot_close"]
)

options["atm"] = (
    options["timestamp"]
    .map(spot_close_map)
    .div(50)
    .round()
    .mul(50)
)

options = options[
    options["opt_strike"].between(
        options["atm"] - 100,
        options["atm"] + 100
    )
]


In [8]:
print("CLEANED SHAPES")
print("Spot:", spot.shape)
print("Futures:", futures.shape)
print("Options:", options.shape)

print("\nOptions per timestamp:")
options.groupby("timestamp").size().describe()


CLEANED SHAPES
Spot: (104595, 6)
Futures: (105196, 8)
Options: (182594, 9)

Options per timestamp:


count    19212.000000
mean         9.504164
std          0.685698
min          5.000000
25%          9.000000
50%         10.000000
75%         10.000000
max         10.000000
dtype: float64

In [9]:
spot.to_csv(PROCESSED_DIR / "nifty_spot_5min.csv", index=False)
futures.to_csv(PROCESSED_DIR / "nifty_futures_5min.csv", index=False)
options.to_csv(PROCESSED_DIR / "nifty_options_5min.csv", index=False)


In [10]:
with open(PROCESSED_DIR / "data_cleaning_report.txt", "w") as f:
    f.write("DATA CLEANING REPORT\n")
    f.write("====================\n\n")

    f.write(f"Spot rows after cleaning: {len(spot)}\n")
    f.write(f"Futures rows after cleaning: {len(futures)}\n")
    f.write(f"Options rows after cleaning: {len(options)}\n\n")

    f.write("Cleaning steps performed:\n")
    f.write("- Unified 5-minute master timeline\n")
    f.write("- Forward-filled OHLC prices\n")
    f.write("- Removed extreme return outliers (Z > 5)\n")
    f.write("- Forward-filled futures open interest\n")
    f.write("- Removed invalid option IV values\n")
    f.write("- Dynamically recalculated ATM strikes\n")
    f.write("- Retained ATM ±2 strikes only\n")


#### TASK 1.3 — Data Merging

In [11]:
# Spot and Futures: one row per timestamp → simple merge
merged = pd.merge(
    spot,
    futures.drop(columns=["contract"], errors="ignore"),  # contract not needed further
    on="timestamp",
    how="inner"
)

# Options: multiple rows per timestamp → aggregate to wide features
options_agg = (
    options
    .pivot_table(
        index="timestamp",
        columns=["opt_strike", "opt_type"],
        values=["opt_ltp", "opt_iv", "opt_open_interest", "opt_volume"]
    )
)

# Flatten multi-index columns
options_agg.columns = [
    f"{val}_{strike}_{otype}"
    for val, strike, otype in options_agg.columns
]

options_agg = options_agg.reset_index()

# Final merge
merged = pd.merge(
    merged,
    options_agg,
    on="timestamp",
    how="inner"
)

print("Merged dataset shape:", merged.shape)
merged.head()


Merged dataset shape: (19212, 2636)


,timestamp,spot_open,spot_high,spot_low,spot_close,spot_volume,fut_open,fut_high,fut_low,fut_close,...,opt_volume_33650_CE,opt_volume_33650_PE,opt_volume_33700_CE,opt_volume_33700_PE,opt_volume_33750_CE,opt_volume_33750_PE,opt_volume_33800_CE,opt_volume_33800_PE,opt_volume_33850_CE,opt_volume_33850_PE
0,2025-01-16 09:15:00,18016.461061,18035.198148,17986.333085,18016.461061,420333.0,18030.567622,18049.319380,18000.416056,18030.567622,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-16 09:20:00,18012.338002,18038.861250,17995.635816,18012.338002,282041.0,18064.344535,18090.944363,18047.594125,18064.344535,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-16 09:25:00,18033.710411,18055.444272,18016.808443,18033.710411,338254.0,18081.221769,18103.012889,18064.275271,18081.221769,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-01-16 09:30:00,18083.578491,18118.754450,18053.506466,18083.578491,116076.0,18106.746245,18141.967270,18076.635694,18106.746245,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-16 09:35:00,18076.319824,18094.912659,18063.795290,18076.319824,163125.0,18112.842409,18131.472811,18100.292570,18112.842409,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
MERGED_DIR = PROJECT_ROOT / "data" / "merged"
MERGED_DIR.mkdir(parents=True, exist_ok=True)

merged.to_csv(MERGED_DIR / "nifty_merged_5min.csv", index=False)

print("Saved: data/merged/nifty_merged_5min.csv")


Saved: data/merged/nifty_merged_5min.csv
